In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import time

In [3]:
with open('letters_source.txt', 'r', encoding = 'utf-8') as f:
    source_data = f.read()
    
with open('letters_target.txt', 'r', encoding = 'utf-8') as f:
    target_data = f.read()

In [4]:
source_data.split('\n')[:10]

['bsaqq',
 'npy',
 'lbwuj',
 'bqv',
 'kial',
 'tddam',
 'edxpjpg',
 'nspv',
 'huloz',
 'kmclq']

In [5]:
target_data.split('\n')[:10]

['abqqs',
 'npy',
 'bjluw',
 'bqv',
 'aikl',
 'addmt',
 'degjppx',
 'npsv',
 'hlouz',
 'cklmq']

In [6]:
def extrtact_character_vocab(data):
    #construct mapping table
    special_words = ['<PAD>','<UNK>','<GO>','<EOS>']
    set_words = list(set([char for line in data.split('\n') for char in line]))
    
    #add the special words in vocab
    int_to_vocab = {idx : word for idx, word in enumerate(special_words + set_words)}
    vocab_to_int = {word : idx for idx, word in int_to_vocab.items()}
    
    return int_to_vocab, vocab_to_int

In [7]:
#mapping table
source_int_to_letter, source_letter_to_int = extrtact_character_vocab(source_data)
target_int_to_letter, target_letter_to_int = extrtact_character_vocab(target_data)

#convert character
source_int = [[source_letter_to_int.get(letter, source_letter_to_int['<UNK>']) 
              for letter in line] for line in source_data.split('\n')]#DISPLAY EACH WORD IN VOCAB IN FORM OF CHAR, 
#THEN TRANSLATE TO INT, UNK is default return value for get in dict
target_int = [[target_letter_to_int.get(letter, target_letter_to_int['<UNK>'])
              for letter in line] + [target_letter_to_int['<EOS>']] for line in target_data.split('\n')]
#<EOS> at the end of every word

In [8]:
source_int[:10]

[[12, 10, 4, 28, 28],
 [18, 8, 23],
 [14, 12, 21, 5, 26],
 [12, 28, 11],
 [22, 7, 4, 14],
 [6, 29, 29, 4, 15],
 [24, 29, 20, 8, 26, 8, 17],
 [18, 10, 8, 11],
 [9, 5, 14, 16, 25],
 [22, 15, 13, 14, 28]]

In [9]:
target_int[:10]

[[4, 12, 28, 28, 10, 3],
 [18, 8, 23, 3],
 [12, 26, 14, 5, 21, 3],
 [12, 28, 11, 3],
 [4, 7, 22, 14, 3],
 [4, 29, 29, 15, 6, 3],
 [29, 24, 17, 26, 8, 8, 20, 3],
 [18, 8, 10, 11, 3],
 [9, 14, 16, 5, 25, 3],
 [13, 22, 14, 15, 28, 3]]

In [10]:
def get_inputs():
    '''
    input tensor of model
    '''
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    

    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return inputs, targets, learning_rate, target_sequence_length, max_target_sequence_length, source_sequence_length

In [11]:
def get_encoder_layer(input_data, rnn_size, num_layers, source_sequence_length, 
                      source_vocab_size, encoding_embedding_size):
    #Encoder embedding
    encoder_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)
    
    #RNN Cell
    def get_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer = tf.random_uniform_initializer(-0.1, 0.1, seed = 2))
        return lstm_cell
    cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    
    encoder_output, encoder_state = tf.nn.dynamic_rnn(cell, encoder_embed_input,
                                                     sequence_length = source_sequence_length, dtype = tf.float32)
    
    return encoder_output, encoder_state    

In [12]:
def process_decoder_input(data, vocab_to_int, batch_size):
    
    ending = tf.strided_slice(data, [0, 0], [batch_size, -1], [1, 1])
    decoder_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)
    
    return decoder_input

In [13]:
def decoding_layer(target_letter_to_ini, decoding_embedding_size, num_layers, rnn_size,
                  target_sequence_length, max_target_sequence_length, encoder_state, decoder_input):
    #1. Embedding
    target_vocab_size = len(target_letter_to_int)
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings, decoder_input)
    
    #2. construct RNN Cell in Decoder
    def get_decoder_cell(rnn_size):
        decoder_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer = tf.random_uniform_initializer(-0.1, 0.1, seed = 2))
        
        return decoder_cell
    
    cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size) for _ in range(num_layers)])
    
    #3. Fully Connected Output
    output_layer = tf.layers.Dense(target_vocab_size, 
                                   kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev = 0.1))
    
    #4. Training Decoder
    with tf.variable_scope("decode"):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        # construct decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder, 
                                                                  impute_finished=True, 
                                                                  maximum_iterations=max_target_sequence_length)
    # 5. Predicting decoder
    # share parameters with training
    with tf.variable_scope("decode", reuse=True):
        # create a constant tensor and copy it as the size of batch_size
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']], dtype=tf.int32), [batch_size], 
                               name='start_tokens')
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                                start_tokens,
                                                                target_letter_to_int['<EOS>'])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                        predicting_helper,
                                                        encoder_state,
                                                        output_layer)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder, impute_finished=True,
                                                                        maximum_iterations=max_target_sequence_length)
    
    return training_decoder_output, predicting_decoder_output

In [14]:
def seq2seq_model(input_data, targets, lr, target_sequence_length, 
                  max_target_sequence_length, source_sequence_length,
                  source_vocab_size, target_vocab_size,
                  encoder_embedding_size, decoder_embedding_size, 
                  rnn_size, num_layers):
    
    # aquire encoder state output
    _, encoder_state = get_encoder_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length,
                                  source_vocab_size, 
                                  encoding_embedding_size)
    
    
    # decoder's output after preprocessing
    decoder_input = process_decoder_input(targets, target_letter_to_int, batch_size)
    
    # state and input to decoder
    training_decoder_output, predicting_decoder_output = decoding_layer(target_letter_to_int, 
                                                                       decoding_embedding_size, 
                                                                       num_layers, 
                                                                       rnn_size,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       encoder_state, 
                                                                       decoder_input) 
    
    return training_decoder_output, predicting_decoder_output

In [15]:
#hyper
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001

In [16]:
# construct graph
train_graph = tf.Graph()

with train_graph.as_default():
    
    # get model's input   
    input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length = get_inputs()
    
    training_decoder_output, predicting_decoder_output = seq2seq_model(input_data, 
                                                                      targets, 
                                                                      lr, 
                                                                      target_sequence_length, 
                                                                      max_target_sequence_length, 
                                                                      source_sequence_length,
                                                                      len(source_letter_to_int),
                                                                      len(target_letter_to_int),
                                                                      encoding_embedding_size, 
                                                                      decoding_embedding_size, 
                                                                      rnn_size, 
                                                                      num_layers)    
    
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

W0924 00:05:27.212854 140441869219648 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0924 00:05:27.213925 140441869219648 deprecation.py:506] From /root/anaconda3/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0924 00:05:27.221324 140441869219648 deprecation.py:323] From <ipython-input-11-78ad1669b7cd>:8: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl)

In [17]:
def pad_sentence_batch(sentence_batch, pad_int):
    #padding
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [18]:
def get_batches(targets, sources, batch_size, source_pad_int, target_pad_int):
    #in order to get batch
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]
        #padding
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))
        
        # record each len
        targets_lengths = []
        for target in targets_batch:
            targets_lengths.append(len(target))
        
        source_lengths = []
        for source in sources_batch:
            source_lengths.append(len(source))
        
        yield pad_targets_batch, pad_sources_batch, targets_lengths, source_lengths

In [19]:
#train and vali
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]
# one batch to validate
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]
(valid_targets_batch, valid_sources_batch, valid_targets_lengths, valid_sources_lengths) = next(get_batches(valid_target, valid_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']))

display_step = 50 # each 50 times print loss

checkpoint = "trained_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
        
    for epoch_i in range(1, epochs+1):
        for batch_i, (targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(
                get_batches(train_target, train_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>'])):
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: sources_batch,
                 targets: targets_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths})

            if batch_i % display_step == 0:
                
                # calculate validation loss
                validation_loss = sess.run(
                [cost],
                {input_data: valid_sources_batch,
                 targets: valid_targets_batch,
                 lr: learning_rate,
                 target_sequence_length: valid_targets_lengths,
                 source_sequence_length: valid_sources_lengths})
                
                print('Epoch {:>3}/{} Batch {:>4}/{} - Training Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_source) // batch_size, 
                              loss, 
                              validation_loss[0]))

    
    
    # save model
    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')

Epoch   1/60 Batch    0/77 - Training Loss:  3.402  - Validation loss:  3.398
Epoch   1/60 Batch   50/77 - Training Loss:  2.887  - Validation loss:  2.889
Epoch   2/60 Batch    0/77 - Training Loss:  2.549  - Validation loss:  2.556
Epoch   2/60 Batch   50/77 - Training Loss:  2.143  - Validation loss:  2.157
Epoch   3/60 Batch    0/77 - Training Loss:  2.012  - Validation loss:  2.010
Epoch   3/60 Batch   50/77 - Training Loss:  1.817  - Validation loss:  1.780
Epoch   4/60 Batch    0/77 - Training Loss:  1.673  - Validation loss:  1.661
Epoch   4/60 Batch   50/77 - Training Loss:  1.558  - Validation loss:  1.487
Epoch   5/60 Batch    0/77 - Training Loss:  1.425  - Validation loss:  1.407
Epoch   5/60 Batch   50/77 - Training Loss:  1.348  - Validation loss:  1.273
Epoch   6/60 Batch    0/77 - Training Loss:  1.246  - Validation loss:  1.218
Epoch   6/60 Batch   50/77 - Training Loss:  1.191  - Validation loss:  1.116
Epoch   7/60 Batch    0/77 - Training Loss:  1.103  - Validation

Epoch  54/60 Batch    0/77 - Training Loss:  0.008  - Validation loss:  0.016
Epoch  54/60 Batch   50/77 - Training Loss:  0.007  - Validation loss:  0.015
Epoch  55/60 Batch    0/77 - Training Loss:  0.007  - Validation loss:  0.016
Epoch  55/60 Batch   50/77 - Training Loss:  0.007  - Validation loss:  0.015
Epoch  56/60 Batch    0/77 - Training Loss:  0.007  - Validation loss:  0.016
Epoch  56/60 Batch   50/77 - Training Loss:  0.006  - Validation loss:  0.015
Epoch  57/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.015
Epoch  57/60 Batch   50/77 - Training Loss:  0.006  - Validation loss:  0.014
Epoch  58/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.015
Epoch  58/60 Batch   50/77 - Training Loss:  0.006  - Validation loss:  0.014
Epoch  59/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.015
Epoch  59/60 Batch   50/77 - Training Loss:  0.005  - Validation loss:  0.014
Epoch  60/60 Batch    0/77 - Training Loss:  0.005  - Validation

In [20]:
def source_to_seq(text):
    '''
    convert source data
    '''
    sequence_length = 7
    return [source_letter_to_int.get(word, 
                                     source_letter_to_int['<UNK>']) for word in text] + [source_letter_to_int['<PAD>']]*(sequence_length-len(text))

In [21]:
input_word = 'zhengjiapengniubi'
text = source_to_seq(input_word)

checkpoint = "./trained_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # load model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(input_word)]*batch_size, 
                                      source_sequence_length: [len(input_word)]*batch_size})[0] 


pad = source_letter_to_int["<PAD>"] 

print('Original INPUT:', input_word)

print('\nSource')
print('  Word Num.:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

print('\nTarget')
print('  Word Num.:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))

W0924 00:06:42.194881 140441869219648 deprecation.py:323] From /root/anaconda3/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Original INPUT: zhengjiapengniubi

Source
  Word Num.:    [25, 9, 24, 18, 17, 26, 7, 4, 8, 24, 18, 17, 18, 7, 5, 12, 7]
  Input Words: z h e n g j i a p e n g n i u b i

Target
  Word Num.:       [4, 12, 24, 17, 9, 7, 7, 26, 7, 18, 18, 8, 8, 5, 25, 3]
  Response Words: a b e g h i i j i n n p p u z <EOS>
